EGM722 Project - Lighting Column Replacements and Drainage Cleaning for MMaRC Network B

# Imports the required packages

In [ ]:
import os
import geopandas as gpd
import matplotlib.pyplot as plt
from cartopy.feature import ShapelyFeature
import cartopy.crs as ccrs
import matplotlib.patches as mpatches
import matplotlib.lines as mline
import folium